# Applied Seismology, GEOS 626, University of Alaska Fairbanks

- script hw_sumatraA_CAN_response.ipynb
- template script for analyzing the amplitude spectra of the Sumatra earthquake
- the example waveform is from CAN (Canberra, Australia) for the 2004 Mw 9.1 Sumatra-Andaman earthquake

In [ ]:
%matplotlib widget

import matplotlib.pylab as plt
import numpy as np
import warnings

from lib_seis import markp_minutes
from lib_seis import sumatra_event
from lib_seis import wf_fft
from matplotlib import dates
from obspy.core import UTCDateTime
from obspy.clients.fdsn import Client

In [ ]:
# script settings

warnings.filterwarnings('ignore')
plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = 8, 6
plt.rcParams['lines.linewidth'] = 1

### Event details
Mw 9.1 2004 Sumatra - Andaman Islands Earthquake \
https://earthquake.usgs.gov/earthquakes/eventpage/official20041226005853450_30

In [ ]:
# fetch and display event details

event = sumatra_event()

print()
for item in event:
    print(f'{item} = {event[item]}')
print()    

In [ ]:
# getting the waveform

client = Client("IRIS")

starttime  = event['origin_time'] - ( 0.5 * 24 * 60 * 60 )           # half a day before origin time
endtime    = event['origin_time'] + ( 9.5 * 24 * 60 * 60)            # 9 and a half days after origin time

st = client.get_waveforms("G", "CAN", "*", "LHZ", starttime, endtime, attach_response=True)
print(st)

In [ ]:
# plotting the seismogram

can = st.copy()                           

t   = can[0].times()                                           # time axis for plotting in seconds
tr  = can[0]

title = f'{tr.id} − starting {tr.stats["starttime"]}'

# plot the seismogram with time in seconds
plt.figure()
plt.plot(t, tr.data, 'b')
plt.title(title)
#plt.ylim([-3E4, 3E4])
plt.xlabel('time [s]')
plt.ylabel('counts');

In [ ]:
# label the x-axis using absolute time

can_dates = st.copy().resample(0.1)

date = can_dates[0].times(type='utcdatetime')                      # time axis for plotting absolute time
date = dates.date2num(date)

# plot the seismogram with respect to absolute time
plt.figure()
plt.plot_date(date, can_dates[0].data, 'b')
plt.title(title)
plt.xlabel('dates')
plt.ylabel('counts')
ax = plt.gca()
ax.xaxis.set_tick_params(rotation=30, labelsize=10)

In [ ]:
# spectral analysis

can.detrend('constant')
can.detrend('linear')
can.taper(max_percentage=0.2, type="cosine")

tr = can[0]

sampling_frequency = tr.stats.sampling_rate
Nyquist_frequency  = sampling_frequency / 2

fft_amp, fft_phase, fft_freq = wf_fft(tr.data, Nyquist_frequency)

C = fft_amp * np.exp( 1j * fft_phase )                     # Fourier transform reconstructed from its amplitude and phase

In [ ]:
# template code for using markp_minutes to identify the periods (in minutes) of spectral peaks

#fig=plt.figure();
#YOUR PLOT COMMAND HERE
#cid = fig.canvas.mpl_connect('button_press_event', markp_minutes) #displays the value 1/x/60 for a selected point (x,y)